**Projeto offline de aprendizado de máquina de ponta a ponta**

---

Neste Notebook você irá criar modelos preditivos para o setor imobiliário.  Usaremos um conjunto de dados do setor imobiliário da Califórnia, baseado no censo de 1990 da cidade. 

As informações mais detalhadas sobre o conjunto de dados podem ser obtidas aqui [nesse artigo](https://www.sciencedirect.com/science/article/abs/pii/S016771529600140X).

Este notebook foi construído baseando-se no [livro do Aurélien Géron](https://www.amazon.com.br/M%C3%A3os-obra-aprendizado-Scikit-Learn-inteligentes/dp/8550815489/ref=asc_df_8550815489/?tag=googleshopp00-20&linkCode=df0&hvadid=379715964603&hvpos=&hvnetw=g&hvrand=6748800514414021109&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=1032060&hvtargid=pla-1390910077420&psc=1) e também através do notebook do Aurélien Géron, [disponível aqui](https://github.com/ageron/handson-ml).




Você verá um panorama geral de alguns dos principais passos de um projeto de machine learning:

1. Enquadrar o problema;

2. Obter os dados;

3. Descobrir e visualizar os dados para obter informações;

4. Preparar os dados para os algoritmos;

5. Selecionar e treinar modelos;

6. Ajustar o modelo.

Para finalizar o ciclo completo de um projeto, seriam ainda necessários:

7. Apresentar sua solução;

8. Lançar, monitorar e manter seu sistema.

Para apresentar a solução seria importante apresentar de maneira organizada e sistematizada as análises que faremos aqui neste notebook. Podería-se ainda criar uma narrativa para que esses resultados fossem apresentados à gestores ou outros profissionais interessados no assunto mas que não são especialistas no assunto.

A parte de lançamento, monitoramente e manuntenção do sistema envolve outras áreas da computação e ao longo do nosso curso estaremos abordando até a parte do deploy na nuvem, isto é, de implantar o modelo para que ele seja consumido por um usuário final. 

Como não faremos o deploy do modelo, estaremos nomeando ele de **offline** apenas para indicar que ele não estará disponível online. 



# Configuração inicial

Vamos começar importando algumas bibliotecas básicas:

*Numpy* - Pacote para computação científica em Python. [Saiba mais.](https://numpy.org/)

*os* - Diversas interfaces para sistema operacional. [Saiba mais.](https://docs.python.org/3/library/os.html)

O NumPy é extramente útil, fornecendo das mais básicas às mais avançadas técnicas de computação científica.



In [ ]:
# Importações comuns
import numpy as np
import os

Vamos agora fixar o sorteio aleatório de números no nosso projeto. Observa que isso é importante para que possamos reproduzir o modelo.

In [ ]:
#Para garantir estabilidade e ser mais fácil reproduzir experimento
seed = 42
np.random.seed(seed)

Vamos agora importar módulo básicos do [matplotlib](https://matplotlib.org/) para plotar figuras. [Confira aqui](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.rc.html) a documentação do matplotlib.rc

In [ ]:
# Para plotar figuras
#Gráficos matplotlib incluídos no notebook, ao lado do código
%matplotlib inline 
import matplotlib as mpl 
import matplotlib.pyplot as plt 

mpl.rc('axes', labelsize=14) 
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

É comum que alarmes sejam disparados quando há algum erro interno ao rodar os códigos. Em geral é importante manter eles ligados pois podem nos ajudar a identificar possíveis erros no código. 

Por hora vamos desligar alguns warnings desnecessários relacionados ao 'internal gelsd'. Você pode conferir essa issue [aqui no GitHub](https://github.com/scipy/scipy/issues/5998). 

In [ ]:
# Ignorar warnings desnecessários (ver SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Enquadar o problema

- Qual o objetivo do problema?

- Como a empresa/cliente pretende usar o produto?

Tais perguntas são importantes pois definirá como você vai abordar o problema, que tipo de algoritmo irá usar e qual o critério utilizado para comparar os modelos (isto é, qual métrica).

Você precisará avaliar se a solução requer uma solução muito complexa, que demandará mais trabalho, tempo e dinheiro, ou se uma solução mais simples será suficiente.

 # Observações importante

- Antes de começar a trabalhar no projeto, verifique todas as hipóteses do sistema, infraestrutura disponível, linguagens de programação que serão utilizada, plataformas, etc. E mais importante: um projeto de machine learning é executado por **pessoas**, conhecer e dialogar com a equipe é indispensável;

- Certifique-se que você dispõe dos dados corretos para construir a solução que o problema exige. Fique atento as informações e à **qualidade dos dados**, isso limitará bastante a parte de modelagem. Alguns dados podem ser inviáveis de serem coletados, seja pelo seu custo ou por tempo limitado do projeto;

- Não aborde, em um primeiro momento, um problema usando a solução mais complexa possível. Otimização prematura é arriscado e pode comprometer o projeto;

- Leve em consideração que os **modelos mais complexos são mais difíceis de manter**, requer estruturas mais sofisticadas (e mais caras) e geralmente requer um corpo técnico mais qualificado - fique atento também às regulamentações dos dados;

- Comece com **protótipos rápidos** e vá conversando com o cliente obtendo retorno sobre as necessidades do produto. Já pensou passar meses desenvolvendo um produto e no final não era o que o cliente queria? A agilidade em fazer protótipos em Python torna essa linguagem muito interessante!

- As nossas visões, opiniões vão mudando com o tempo, então é natural que o cliente (e você!) vá amadurecendo ao longo do processo. **Comunicação** é a palavra chave.




# Obtendo os dados

Nesta etapa você deverá obter os dados necessários ao problema. A função definida à serguir faz um htpp request no link, faz download do arquivo zip e após isso extrai os dados, salvando no diretório "datasets/housing".

In [ ]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True) #Cria diretorio
    tgz_path = os.path.join(housing_path, "housing.tgz") #caminho do arquivo
    urllib.request.urlretrieve(housing_url, tgz_path) #request data
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close() #Importante!!!

In [ ]:
fetch_housing_data() #Cria diretório datasets/housing no espaço de trabalho

Vamos importar a biblioteca do pandas que é extremamente útil para lidar dataframs (tabela de dados). Vamos carregar os dados no diretório criado, criando um ***pandas frame*** que conterá as informações do arquivo housing.csv

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path) #Função do pandas para carregar arquivo CSV

In [ ]:
housing = load_housing_data()

# Conheçendo os dados

Vamos começar visualizando as 5 primeiras linhas do dataframe

In [ ]:
housing.head()

Observa que todas as 5 primeiras amostras tem ocean_proximity = 'NEAR BAY', o que pode indicar que os dados estejam de certa forma ordenados.  

Vamos fazer então o seguinte: vamos coletar uma amostra contendo 10 instâncias para visualizar os nossos dados. Observa que esse processo envolve uma aleatoriedade, daí a importância de fixar random_state em um certo valor caso você tenha interesse em reproduzir o experimento.

In [ ]:
housing.sample(n = 10, random_state = seed)

Olha que interessante! Se você usar a seed fixada no início (42), você observará que tem dados faltantes no número total de quartos.  

Vamos agora nos informar a respeito das variáveis do problemas

In [ ]:
housing.info() #Rápida descrição dos dados

É importante também saber **como** esse dataset foi **construído**. 

Primeiro, usou-se dados brutos do censo de 1990 da Califórnia. 

* Calculou-se os centróides de cada quarteirão da Califórnia, medido em latitude e longitude.

* Foram excluídos todos os quarteirões que tinha entradas faltantes.

As características (**features**) são as seguintes:


1. longitude: longitude do centro do quarteirão;

2. latitude: latitude do centro do quarteirão;

3. housing_median_age: idade mediana de uma casa dentro de um quarteirão; 
4. total_rooms: número total de quartos em uma quadra;

5. total_bedrooms: número total de quartos em uma quadra;

6. population: número total de pessoas residentes em um quarteirão;

7. households: número total de famílias em um quarteirão;

8. median_income: renda mediana para famílias em um quarteirão de casas (medida em dezenas de milhares de dólares);

9. median_house_value : valor médio da casa para famílias em um bloco (medido em dólares americanos);

10. ocean_proximity: localização da quadra em relação ao mar/oceano.

As informações são do [artigo](https://www.sciencedirect.com/science/article/abs/pii/S016771529600140X).

Todos as características são numéricas, exceto a proximidade do oceano (último atributo). Nesse caso, sabemos que o último campo é na verdade do tipo texto, embora o Python tenha carregado como um objeto genérico.

In [ ]:
housing["ocean_proximity"].value_counts()

Como podemos observar, a característica "proximidade do oceano" é um atributo 
categórico.

Vamos agora extrair algumas medidas resumo do nosso conjunto de dados.

In [ ]:
housing.describe() #Medidas resumo

Alguns histogramas também são úteis para compreender o problema

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

#Separando o conjunto de dados

Se quisermos utilizar um conjunto de teste para realizar uma estimativa "não enviesada" do modelo final é importante já separarmos o conjunto de treino e teste desde já.

Observa que a função train_test_split implentada no scikit-learn tem como padrão shuffle = True. Isto quer dizer que ele irá embaralhar os dados e então fara a divisão do conjunto de dados em treino e teste. 

<font color='red'>É importante que os dados sejam embaralhados pois é comum que exista algum tipo de ordenação nos dados, de forma que se você não embaralhar os dados estará introduzindo tendencias ou vieses que não existem nos dados reais. </font>


In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, 
                                       test_size=0.2, #20% para teste
                                       random_state=seed)

Essa divisão no conjunto de dados é até então puramente aleatória. Será que esse tipo de divisão é a mais indicada?

Vejamos novamente as medidas resumo da renda mediana:

In [ ]:
housing["median_income"].describe()

In [ ]:
mean = np.mean(housing["median_income"])
std = np.std(housing["median_income"])

Lembrando que "median_income" é a renda mediana para famílias em um quarteirão de casas (medida em dezenas de milhares de dólares). Nesse caso, a **maior parte dos quarteirões tem renda mediana entre 20 mil e 58 mil dólares***.


***Observação:** mean ~= 3.87, corresponde a uma renda anual mediana de 38 mil e 700 dólares. Neste caso, calculando o intervalo [mean - std, mean + std], em um modelo gaussiano teríamos aproxidamente 68% da distribuição dos dados. Daí a afirmação.


Vamos agora fazer uma estratificação da renda, pois é importante ter um número suficiente de instâncias para cada estrato no conjunto de dados (treino e testes), do contrário pode ser que os nossos dados fiquem enviasados, não representando adequadamente a população. Em uma amostra suficientemente grande isso não seria um problema.

Dependendo do problema, a questão da discussão do tipo de amostragem deve ser discutida com um **estatístico**.

Vamos supor que tenhamos feito esse processo, de discutir com um estatístico a respeito do problema e foi nos informado que é importante separar a renda dos quarteirões em estratos para abordar o problema adequadamente.

Vamos dividir a renda em 5 estratos, de 15 em 15 mil doláres. Não há nenhuma mágica nessa escolha, senão a questão da facilidade. O correto seria, mais uma vez, usar informações sociais discutidas com o estatístico

Faremos isso criando uma nova feature no nosso dataset. Esse processo de criar novas categorias a partir do conhecimento do problema é chamado de **feature engineering** e abordaremos melhor mais à frente no nosso projeto.

In [ ]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

housing["income_cat"].hist()

In [ ]:
housing["income_cat"].value_counts()

Pronto! Agora vamos fazer uma <font color='red'>amostragem estratificada</font> com base nas categorias da renda.  

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)

for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

Acabamos de criar novos conjuntos de treino e de teste, que chamamos de <font color='red'>strat_train_set </font> e <font color='blue'>strat_test_set</font>.

 Estes conjuntos devem respeitar a estratificação que introduzimos baseada em "median_income" representado na nova variável categórica "income_cat".

 Vejamos se funcionou:

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set) #Proporção de cada categoria em strat_test_set

In [ ]:
housing["income_cat"].value_counts() / len(housing) #Proporção de cada categoria em housing

Podemos agora comparar com a <font color='blue'> amostragem aleatória </font>:

In [ ]:
#Função para calcular as proporções das categorias da característica "income_cat"
def income_cat_proportions(data): 
    return data["income_cat"].value_counts() / len(data)

Agora vamos gerar novamente conjunto de teste e treino, mas usando amostragem aleatória.

In [ ]:
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=seed)

Vamos criar o nosso novo dataframe e visualizar os resultados:

In [ ]:
compare_props = pd.DataFrame({
    "Geral": income_cat_proportions(housing),
    "Estratificado": income_cat_proportions(strat_test_set),
    "Aleatorio": income_cat_proportions(test_set),
}).sort_index()

compare_props["Aleatório %erro"] = 100 * compare_props["Aleatorio"] / compare_props["Geral"] - 100
compare_props["Estratificado %erro"] = 100 * compare_props["Estratificado"] / compare_props["Geral"] - 100

compare_props

Contentes com os resultados, não podemos esquecer de <font color='red'>remover</font> o atributo "income_cat" dos conjuntos strat_train_set e strat_test_set. Na verdade, ele era apenas um intermediário, afinal de contas as informações dessa caracaterísticas já estão presentes em "median_income".

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Visualização da estrutura de dados

Vamos agora visualizar os nossos dados. Precisamos ter certeza que não vamos visualizar dados do conjunto de teste, para evitar enviesamento de conclusões. 

De um ponto de vista mais técnico, devemos evitar o **snooping bias**.

In [ ]:
housing = strat_train_set.copy() #Importante criar uma cópia! 

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")

Vamos melhorar a visualição usando o parâmetro <font color='red'>alpha</font>, observe:

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

Interessante! Agora fica mais evidente a concentração dos agrupamentos!

De qualquer forma devemos voltar a nossa atenção ao objetivo: <font color = 'red'> preços do setor imobioliário. </font> 

No código a seguir o parâmetro "s" significa "size", tamanho em inglês. Escolhendo "s" como sendo a característica população, quanto maior o disco representa uma população maior.

O parâmetro "c" significa "color", ou cor. Esse é na verdade o que queremos saber!

O paramêtro colorbar = True indica que queremos visualizar a barra lateral informando as intensidades da cor, ou seja, do parêmetro "c".



In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
    s=housing["population"]/100, label="population", figsize=(10,7),
    c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False) #sharex=false é só pra corrigir um bug de display https://github.com/pandas-dev/pandas/issues/10611
plt.legend()

A visulização dos dados indicam que regiões litorâneas tendem a possuir um valor mais alto. Talevz a densidade populacional também possa ser algo relevante.

Vamos então investigar essas hipóteses através da correleção estatística:

In [ ]:
corr_matrix = housing.corr() #Matriz de correlações

In [ ]:
corr_matrix #vamos ver a estrutura

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False) #Ordenar valores em sentido decrescente

É conveniente usar o scatter_matrix do pandas. Essa função plota cada característica em relação a outra. No nosso exemplo, teríamos 121 possibilidades.

Mas claro que não faremos isso e vamos então selecionar algumas que parecem ser mais significativas:

**Dica**: Vamos aproveitar e revisar alguns [conceitos básicos de estatística](http://geam.paginas.ufsc.br/files/2020/02/Estatistica_Basica.pdf).



In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))

**OBS:** Na diagonal principal da plotagem anterior não temos atributo v.s. atributo, mas sim o histograma da característica.

Vimos antes que a característica que tinha maior correlação com o valor mediano de casas em um bairro era o salário mediano. Então vamos plotar para estudar a relação entre ambos:

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value",
             alpha=0.1)
plt.axis([0, 16, 0, 550000])

Informações desta plotagem: 

1.   Correlação é forte;

2.   Há um valor limiar de 500.000 para os valores (medianos) das casas. Por quê?

3. Há também outras linhas horizontais. Por que elas são importantes?

Uma abordagem possível seria excluir os dados correspondentes a esses casos.

#Feature Engineering

Além das colunas que o conjunto de dados nos oferece, podemos tentar construir novas características <font color = "red">**construídas de maneiras não linear**</font> com as características existentes.

De maneira geral, essa etapa requer conhecimento específico da área na qual se esta trabalhando. Daí a importância da presença de um especialista no assunto para auxiliar no projeto. 

A seguir, vamos construir algumas novas features que são mais ou menos lógicas.

In [ ]:
#Nova feature: Número de cômodos por familia (média)
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]

#Nova feature: quartos/cômodos
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]

#Nova feature: população/agregado familiar
housing["population_per_household"]=housing["population"]/housing["households"]

Vejamos agora a matriz de correlação de housing:

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

Aparentemente, casas com uma baixa proporção de quartos por cômodos tendem a ser mais caras. O número de cômodos por família é muito mais informativo que o número total de quartos em um quarteirão.

Vejamos o gráfico:

In [ ]:
housing.plot(kind="scatter", x="rooms_per_household", y="median_house_value",
             alpha=0.2)
plt.axis([0, 5, 0, 520000])
plt.show()

Vamos ver novamente as medidas resumos considerando as novas features!

In [ ]:
housing.describe()

# Preparar os dados para os algoritmos de Machine Learning

Precisamos incialmente retirar os rótulos do conjunto <fon color='blue'> strat_train_set </font> (mais a frente ficará claro).

Para isso, vamos usar o método drop:

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1) # O método drop cria cópia sem a coluna em questao
housing_labels = strat_train_set["median_house_value"].copy() #salvando uma cópia

**OBS:** Ao longo desta seção estaremos chamando as features de treinamento como "housing". Atenção neste ponto para não confundir com o dataset inteiro. Isto é,
 tudo o que nos faremos aqui será feito somento no conjunto de treinamento!

A partir de agora vamos partir para etapa de <font color='blue'>**limpeza de dados!**</font>

Vamos começar verificando se temos dados falantes:

In [ ]:
#housing.isnull().any(axis=1) verifica quais linhas possuem alguma célula null
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head() 
sample_incomplete_rows

In [ ]:
sample_incomplete_rows

Possuímos basicamente três abordagens possíveis para lidar com os dados faltantes:

1. Excluir os quarteirões com dados faltantes;

2. Excluir toda coluna de total_bedrooms, já que é o único atributo que apresenta dados faltantes;

3. Definir algum valor para substituir total_bedrooms.

In [ ]:
sample_incomplete_rows.dropna(subset=["total_bedrooms"])    # opção 1

In [ ]:
sample_incomplete_rows.drop("total_bedrooms", axis=1)       # opção 2

Opção 3: preenchendo com algum valor - nesse caso, usaremos a mediana.

Usaremos a mediana pois queremos alguma medida simples para corrigir os dados faltantes. Ao mesmo tempo, quando comparada com a média, a mediana é mais robusta a outliers o que a torna bastante interessante.

É claro que existem técnicas mais sofisticadas, por exemplo, há [livros](https://www.amazon.com.br/Statistical-Analysis-Missing-Probability-Statistics-ebook/dp/B07Q25CNSD/ref=sr_1_1?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=Statistical+Analysis+with+Missing+Data&qid=1610115793&s=digital-text&sr=1-1) inteiros sobre o assunto.

In [ ]:
median = housing["total_bedrooms"].median()
sample_incomplete_rows["total_bedrooms"].fillna(median, inplace=True) # opção 3
sample_incomplete_rows

Se escolhermos a opção 3, devemos calular a mediana (ou qualquer outra medida que seja justificável) no <font color="red">**conjunto de treinamento**</font> e usá-lo para preencher os valores faltantes neste, mas precisamos <font color="blue">**salvar**</font> esse valor calculado.

Você precisar desse valor para mais tarde aplicar no conjunto de teste, que deverá ter seus dados faltantes corrigidos seguindo o mesmo parâmetro do conjunto de treino.

**AVISO**: No Scikit-Learn 0.20, a classe `sklearn.preprocessing.Imputer` 
foi substituida pela classe `sklearn.impute.SimpleImputer`. Então, é conviniente verificar qual versão o computador em questão está usando:

In [ ]:
try:
    from sklearn.impute import SimpleImputer # Scikit-Learn 0.20+
    print("Scikit-Learn 0.20+")
except ImportError:
    from sklearn.preprocessing import Imputer as SimpleImputer
    print("Scikit-Learn antes do 0.20")

imputer = SimpleImputer(strategy="median")

Vamos novamente revisar o nosso dataset...

In [ ]:
housing

Ainda temos a última coluna que não é numérica! 

A princípios, grande parte dos algoritmos de machine learning no computador preferem os dados representados numericamente!

In [ ]:
housing_num = housing.drop('ocean_proximity', axis=1)
# Derrubando a coluna "ocean_proximity"
# alternativa: housing_num = housing.select_dtypes(include=[np.number])

Agora vamos ajudar o nosso objeto imputer com o nossos dados:

In [ ]:
imputer.fit(housing_num) 

Aqui, o imputer simplesmente calculou a mediana no conjunto de dados.

Vejamos algumas informações sobre o nosso objeto imputer:

In [ ]:
imputer.statistics_

Vamos verificar que isto é, na verdade, a mesma coisa que calcular manualmente a mediana de cada atributo:

In [ ]:
housing_num.median().values

**Mas não seria apenas o atributo total_bedrooms que estava com valores faltantes?** 

Vamos precisar de todas as informações do imputer? Isto é, vamos precisar da mediana de todas as variáveis?

<font color='red'> **Não podemos, a princípio, afirmar que o mesmo padrão vai ser repetir na generalização do modelo!** </font>

Certo, mas e se dermos uma espiadinha no conjunto de testes?

Não devemos fazer isso por vários motivos. 

1. Corremos o risco de colocar vieses no nosso modelo (assumir que apenas "total_bedrooms" terá colunas com dados faltantes em todos os cenários possíveis é um deles);

2. Devemos ter sempre em mente que o conjunto de teste é no fundo uma simulação para testarmos o poder de generalização do algoritmo - devemos fazer todas as nossas análises e otimizações somente no conjunto de treinamento e então aplicar o modelo final uma única vez no conjunto de teste!



Vamos agora finalmente <font color = 'blue'> transformar </font> o nosso conjunto de dados, aplicando, efetivamente, o valor calculado da mediana nos dados faltantes:

In [ ]:
X = imputer.transform(housing_num) #numpy array

Vamos visualizar o conjunto X

In [ ]:
X

Se você se sentir mais confortável, pode transformar o conjunto X em um dataframe:

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns, #importante informar nome das colunas
                          index=housing.index) #DataFrame Pandas

Vejamos como é este dataframe:

In [ ]:
housing_tr.head()

Agora devemos tratar a variável categórica`ocean_proximity'!

Lembre que esta é uma variável muito importante no nosso problema: ela demonstrava uma boa correlação com o preço mediano das casas.

Vamos novamente visualizar os dados para relembrar:

In [ ]:
housing_cat = housing[['ocean_proximity']]
housing_cat.head(10)

Agora vamos usar um processo chamado de codificação. Vamos transformar as nossas variáveis categóricas em números!

<font color = 'red'>**Agora é um bom momento para olhar o noteobok de [apoio](https://github.com/edsonjunior14/mlcourse/blob/master/material_apoio_codificacao.ipynb) :**)</font>


**OBS**: O código a seguir é apenas devido a atualização da classe OrdinalEnconder()

In [ ]:
try:
    from sklearn.preprocessing import OrdinalEncoder
    print("Scikit-Learn >= 2.0")
except ImportError:
    from future_encoders import OrdinalEncoder # Scikit-Learn < 0.20
    print("O teu Scikit-Learn tá antiguinho mô quirido")

Na função a seguir, precisamos instanciar um objeto ordinal_encoder. 

Depois, usamos fit_transform para executa duas operações:

1. Método fit irá ajustar os parâmetros (mapeamento, por exemplo, quais são as variáveis categóricas); 

2. Método transform irá transformar os dados;

3. fit_transform(dados) irá ajustar parâmetros e transformar os dados.


In [ ]:
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)

Uma alternativa mais prolixa teria sido escrever:

original_encoder.fit(housing_cat)

housing_cat_encoded = original_enconder.transform(housing_cat)


Vejamos que tipo de objeto é housing_cat_encoded:

In [ ]:
type(housing_cat_encoded)

Vamos ver agora os 10 primeiros valores desse numpy array:

In [ ]:
housing_cat_encoded[:10]

Vamos relembrar também as categorias do nosso problema:

In [ ]:
ordinal_encoder.categories_

**Veja!**

O objeto ordinal_encoder foi construiído assim:

ordinal_encoder = OrdinalEncoder() 

e depois fizemos o seguinte:

housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)

<font color = "red">**Aqui não apenas definimos quem é "housing_cat_encoded" como também inserimos informações no objeto ordinal_encoder!** </font>

Apesar dos nossos esforços, temos um grave problema na nossa codificação, veja novamente: 

In [ ]:
housing_cat_encoded[:10]

In [ ]:
housing_cat[:10]

Cada variável categórica foi transformada em número!

Mas será que a princípio, podemos comparar uma variável categórica com outra?

Quem é maior: NEAR OCEAN ou NEAR BAY? 

Bem, é difícil responder. Mas é isso que a nossa codificação implicítacamente está fazendo ao colocar os valores 0, 1, 2, 3 ou 4 para cada variável categórica. 

<font color="red"> **Para lidar com essa situação precisamos então de outra abordagem!**</font>

In [ ]:
try:
    from sklearn.preprocessing import OrdinalEncoder # gera um ImportError se Scikit-Learn < 0.20
    from sklearn.preprocessing import OneHotEncoder
except ImportError:
    from future_encoders import OneHotEncoder # Scikit-Learn < 0.20

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

Epa! Agora temos uma matriz SciPy ao invés de um Numpy array! 

<font color = "red">Por que será?</font>

Por padrão a classe `OneHotEncoder` retorna uma matriz (array) esparso, mas podemos transformá-la convertendo em uma matriz chamando o método `toarray()`:

In [ ]:
housing_cat_1hot.toarray()

Alternativamente, podemos colocar `sparse=False` ao criar o objeto `OneHotEncoder`:

In [ ]:
cat_encoder = OneHotEncoder(sparse=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
cat_encoder.categories_

Vamos criar um transformador customizado para adicionar atributos extras 

**OBS**:
- Vamos criar um código para o processo manual feito na etapa de Feature Engineering. 

- Vai nos ajudar a criar um pipeline mais a frente.

In [ ]:
housing.columns

Alternativamente, você pode usar a função da classe `FunctionTransformer` que permite você criar rapidamente um 
transformador baseado em uma função de transformação! 

In [ ]:
from sklearn.preprocessing import FunctionTransformer

# get the right column indices: safer than hard-coding indices 3, 4, 5, 6
rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

def add_extra_features(X, add_bedrooms_per_room=True):

    rooms_per_household = X[:, rooms_ix] / X[:, household_ix]

    population_per_household = X[:, population_ix] / X[:, household_ix]

    if add_bedrooms_per_room:
        bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]

        return np.c_[X, rooms_per_household, 
                     population_per_household,
                     bedrooms_per_room]
    else:
        return np.c_[X, rooms_per_household, population_per_household]

attr_adder = FunctionTransformer(add_extra_features, 
                                 validate=False,
                                 kw_args={"add_bedrooms_per_room": False})

housing_extra_attribs = attr_adder.fit_transform(housing.values)

#Vale colocar validate=False já que os dados não possuem valores não-float
#validate=false é valor padrão a partir do Scikit-Learn 0.22.

In [ ]:
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns= list(housing.columns) + ["rooms_per_household", 
                                      "population_per_household"],
    index=housing.index)

housing_extra_attribs.head()

Agora vamos construir um "[pipeline](https://scikit-learn.org/stable/modules/compose.html#pipeline)" (tradução literal: gasoduto) para pré-processar os atributos numéricos.

A ideia do pepeline é aplicar, nesta ordem, as seguintes transformações:

*   Dados faltantes são imputadas
*   Novas features são adicionadas (feature engineering)
*   As features são normalizada para que fiquem escaladas

O pipeline será aprendido no conjunto de treino e depois será aplicado, usando as regras aprendidas no treinamento, no conjunto de teste.


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler  #StandardScaler serve para fazer a reescalar das variáveis

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', FunctionTransformer(add_extra_features, validate=False)),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
housing_num_tr

In [ ]:
try:
    from sklearn.compose import ColumnTransformer
except ImportError:
    from future_encoders import ColumnTransformer # Scikit-Learn < 0.20

Agora devemos acrescentar o codificador no nosso pipeline!

In [ ]:
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

#Este é o pipeline completo!
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs), #um pipeline dentro do outro
        ("cat", OneHotEncoder(), cat_attribs),
    ]) 

""" Lembrando: num_pipeline é o pipeline que transforma variavéis numéricas

num_pipeline = Pipeline([
      ('imputer', SimpleImputer(strategy="median")),
      ('attribs_adder', FunctionTransformer(add_extra_features, validate=False)),
      ('std_scaler', StandardScaler()),
    ])
"""

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
housing_prepared

In [ ]:
housing_prepared.shape

Agora finalmente temos os nossos dados de treinamento pré-processados, assim como já temos um modelo de limpeza e tratamento de dados implentado que poderá ser aplicado no conjunto de teste.

# Selecionar e treinar um modelo

Vamos começar com um modelo simples: Regressão Linear!

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels) 
#Ei Regressão linear, encontre os parâmetros que melhor aproxima os dados

Vamos agora testar o nosso pipeline de pré-processamento em algumas instâncias de treino.

- Observe que após os nossos esforços em apenas uma linhas conseguimos pré-processar os dados!

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data) #Full pipeline

print("Predictions:", lin_reg.predict(some_data_prepared))

Vamos comparar agora com os valores reais:


In [ ]:
print("Labels:", list(some_labels))

In [ ]:
some_data_prepared

Agora vamos usar métricas para ver o quão bom foi o modelo:

In [ ]:
from sklearn.metrics import mean_squared_error as MSE

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = MSE(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse) #Não é necessariamente obrigatório
lin_rmse

In [ ]:
from sklearn.metrics import mean_absolute_error as MAE

lin_mae = MAE(housing_labels, housing_predictions)
lin_mae

Essse modelo ainda não parece ser adequado!

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state= seed)
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = MSE(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

O quê? Erro zero? Aqui aconteceu o que nós chamamos de sobreajuste! Desconfie sempre quando o erro do teu modelo for zero. Isso não acontece na prática. O que indica que precisamos encontrar técnicas mais robustas para availiar os nossos modelos. 

Lembrando que o conjunto de teste deve ser usado apenas ao **final** do processo.

No próximo bloco abordaremos uma maneira mais adequada de usar o conjunto de treinamento para avaliar os nossos modelos.



#Avaliação de modelo

Até agora estamos treinando um modelo no conjunto de treinamento e testando nele mesmo, o que não parece ser uma estratégia muito adequada.

 Faremos então o seguinte: vamos separar o conjunto de treinamento em k = 10 pedaços (folds) e fazemos então um loop:

*   Para cada fold:
  1.   Treine o seu modelo no conjunto formado por: treino - fold
  2.   Teste o seu modelo no fold

* Ao final, calcule uma média dos k testes anteriores

Esse processo é o que chamamos de [validação cruzada](https://scikit-learn.org/stable/modules/cross_validation.html#)!



In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10) 

#cv = 10 é número de pedaços

tree_rmse_scores = np.sqrt(-scores)

**OBS:** Os recursos da validação cruzada no Scikit-Learn esperam uma função de utilidade (mais alta é melhor) ao invés de uma função custo (mais alta é pior). Assim a função de pontução é oposto à função custo (negativa). Por isso o np.sqrt(-scores) no código acima.

Vejamos os resultados:

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

Vamos ver agora para o nosso modelo de regressão linear:

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Note que o modelo de árvore de decisão está se sobreajustando aos dados demasiadamente, que acaba sendo pior que a regressão linear!

Vamos tentar outro modelo que veremos mais adiante no curso: "Florestas aleatórias" para regressão. 

Observe que o RandomForestRegressor é uma técnica de **regressão não linear** (assim como as árvores de decisão)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg.fit(housing_prepared, housing_labels) #Treinar modelo

In [ ]:
housing_predictions = forest_reg.predict(housing_prepared) #Predizer
forest_mse = MSE(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Os resultados são melhores! 

Entretanto, ainda observe que a pontuação no conjunto de treino ainda é muito menor do que no conjuntos de validação, o que significa que o modelo ainda está se sobreajustando ao conjunto de treinamento.

Possíveis soluções:
- Simplificar o modelo;
- Regularizar o modelo;
- Obter mais dados de treinamento (hard).

# Ajustando e selecionando modelo

Vamos usar o **[Grid Search](https://scikit-learn.org/stable/modules/grid_search.html#exhaustive-grid-search)** (busca em grades) para buscar melhores parâmetros para a nossa floresta aleatória. 

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # Vamos tentar 12 = 3x4 combinação de parâmetros
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # Tentar 6 = 2×3 combinações do bootstrap no modo 'Falso'
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=seed)

# Vamos treinar com 5-folds, então temos (12+6)*5=90 rodadas de treinamento!!!

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

A melhor combinação de parâmetros encontrada:

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

Vamos olhar a pontuação de cada hiperparâmetro testado ao longo do gridSearch:

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
pd.DataFrame(grid_search.cv_results_)

Uma abordagem alternativa ao GridSearch é usar o [RandomizedSearchCV](https://scikit-learn.org/stable/modules/grid_search.html#randomized-parameter-optimization). Essa nova ferrramenta de busca é indicada para quando deseja-se buscar hiperparâmetros com um número elevado de combinaçoes.

- É usada da mesma maneira que o GridSearch, mas ao invés de tentar todas as combinações ela selaciona um valor aleatório para cada hiperparâmetro em cada iteração e avalia um número de combinações aleatórias;

- Se você permitir muitas iterações (por exemplo, mais de 1000), ela irá explorar 1000 combinações diferentes de hiperparâmetros.



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=seed)

rnd_search = RandomizedSearchCV(forest_reg,
                                param_distributions=param_distribs,
                                n_iter=10,
                                cv=5, 
                                scoring='neg_mean_squared_error', 
                                random_state=seed)

rnd_search.fit(housing_prepared, housing_labels)

Vejamos os resultados:

In [ ]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

Vejamos as características mais importantes! (feature das florestas aleatórias)

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

Pouco informativa... vejamos dessa forma:

In [ ]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"] #Importância de ter salvo
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

#Modelo e teste final

Após todas as etapas anteriores, podemos fazer o teste final do nosso modelo:

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = MSE(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [ ]:
final_rmse

# Depois? 


*   Elabore uma apresentação e construa uma narrativa para apresentar os resultados;

*   Lance, monitore e mantenha seu sistema:

  1.  Preparar solução para produção;

  2.  Código de monitoramente para verificar com certa frequência o desempenho;

  3. Observe com atenção a qualidade do sinal de entrada do sistema. É importante que você mantenha em dia a qualidade dos dados oferecidos ao modelo;

  4.  Lembre que as tendências vão mudando e já que grande parte dos dados são gerados a partir da atividade humana (acessos a site, uso de energia elétrica, hábitos de saúde de uma população, novas tecnologias são construídas, etc)  é natural que os dados passem a ter comportamento distintos com os passar do tempo. Portanto, fique atendo pois modelos de machine learning tendem a perder performance com o tempo;

  5. Idealmente, você automatiza a etapa de coleta de dados, preparação & transformação, escolha de melhor modelo e atualização na nuvem. Se o dedo da estística estiver coçando você pode gerar relatórios automatizados de análise exploratória, de forma que uma equipe compentente possa acompanhar os resultados.